# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
17

17

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [74]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from timeit import default_timer as timer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [7]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [8]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [26]:
df = df[(df.job != 'unknown') & (df.marital != 'unknown') & (df.education != 'unknown') & (df.default != 'unknown') & (df.housing != 'unknown') & (df.loan != 'unknown')]
print(f"Cleaned rows: {df.shape[0]}")

Cleaned rows: 30488


In [60]:
fraction_subscribed = df[df.y == 'yes'].shape[0] / df.shape[0]
print(f"Percentage of subscriptions: {100 * fraction_subscribed:.2f}%")

Percentage of subscriptions: 12.66%


#### Feature Encoding
* One Hot Encoding: job, marital, default, housing, loan, contact, month, day_of_week, poutcome, y
* Ordinal Encoding: education
* Pass through the rest

#### Dropping unknown values
* Drop any columns where categorical value is 'unknown'. This will reduce the total dataset to 30,488 rows which should be enough for analysis.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

#### Business Objective
The business objective is to create a model that predicts which customers would be likely to subscribe to a term deposit. Contacting customers who are unlikely to respond positively can result in customer dissatisfaction and wasted resources. Therefore the marketing campaigns can save time, money, and user trust by targeting customers that are likely to convert.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [42]:
ordinal_ohe = make_column_transformer(
      (OrdinalEncoder(categories = [['illiterate','basic.4y','basic.6y','basic.9y','high.school','professional.course','university.degree']]), ['education']),                   
      (OneHotEncoder(drop = 'if_binary'), ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y']),
      remainder = 'passthrough',
      verbose_feature_names_out = False
)
df_encoded = pd.DataFrame(ordinal_ohe.fit_transform(df), columns=ordinal_ohe.get_feature_names_out())
df_encoded.head()

,education,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,...,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,56.0,261.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,37.0,226.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
2,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,40.0,151.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,56.0,307.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0
4,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,59.0,139.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.857,5191.0


In [44]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30488 entries, 0 to 30487
Data columns (total 48 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   education             30488 non-null  float64
 1   job_admin.            30488 non-null  float64
 2   job_blue-collar       30488 non-null  float64
 3   job_entrepreneur      30488 non-null  float64
 4   job_housemaid         30488 non-null  float64
 5   job_management        30488 non-null  float64
 6   job_retired           30488 non-null  float64
 7   job_self-employed     30488 non-null  float64
 8   job_services          30488 non-null  float64
 9   job_student           30488 non-null  float64
 10  job_technician        30488 non-null  float64
 11  job_unemployed        30488 non-null  float64
 12  marital_divorced      30488 non-null  float64
 13  marital_married       30488 non-null  float64
 14  marital_single        30488 non-null  float64
 15  default_yes        

In [50]:
X_bank_info = df_encoded.iloc[:, :18] # Keep first 17 columns, the encoded bank information.
y = df_encoded['y_yes']

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [82]:
# Stratified to get an even split of positive responses since the classes are unbalanced (12% acceptance rate).
X_train, X_test, y_train, y_test = train_test_split(X_bank_info, y, test_size=0.25, random_state=42, stratify=y)
print("Stratified training and test splits have roughly the same subscription rate")
train_subscription_rate = y_train[y_train == 1].shape[0] / y_train.shape[0]
print(f"Training subscription rate {train_subscription_rate}")
test_subscription_rate = y_test[y_test == 1].shape[0] / y_test.shape[0]
print(f"Test subscription rate {test_subscription_rate}")

Stratified training and test splits have roughly the same subscription rate
Training subscription rate 0.12656345666054403
Test subscription rate 0.12660718971398582


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [84]:
# The baseline predicts the majority class which is 'no'. 
# The score for the baseline prediction is equal to the fraction of test data that said no.
baseline_accuracy = 1 - test_subscription_rate
print(f"Baseline accuracy: {baseline_accuracy:.5f}")

Baseline accuracy: 0.87339


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [70]:
lr = LogisticRegression().fit(X_train, y_train)

### Problem 9: Score the Model

What is the accuracy of your model?

In [71]:
lr.score(X_test, y_test)

0.8733928102860141

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [77]:
def measure_model(model, model_name, result, X_train_, y_train_, X_test_, y_test_):
    start = timer()
    model.fit(X_train_, y_train_)
    end = timer()
    result['Model'].append(model_name)
    result['Train Time'].append(end - start)
    result['Train Accuracy'].append(model.score(X_train_, y_train_))
    result['Test Accuracy'].append(model.score(X_test_, y_test_))

In [85]:
result_dict = {'Model': [], 'Train Time': [], 'Train Accuracy': [], 'Test Accuracy': []}
measure_model(KNeighborsClassifier(), 'KNN', result_dict, X_train.values, y_train, X_test.values, y_test) # Bug workaround https://github.com/scikit-learn/scikit-learn/issues/26768#issuecomment-1676406055
measure_model(LogisticRegression(), 'Logistic Regression', result_dict, X_train, y_train, X_test, y_test)
measure_model(DecisionTreeClassifier(), 'Decision Tree', result_dict, X_train, y_train, X_test, y_test)
measure_model(SVC(), 'SVM', result_dict, X_train, y_train, X_test, y_test)
test_df = pd.DataFrame(result_dict).set_index('Model')
test_df

,Train Time,Train Accuracy,Test Accuracy
Model,,,
KNN,0.002911,0.859704,0.857911
Logistic Regression,0.104046,0.873437,0.873393
Decision Tree,0.028500,0.875098,0.871031
SVM,4.217083,0.873437,0.873393


#### Model Comparisons
The best results of these models only match the baseline (always predict no) which scored 0.87339.

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

##### Questions